# Vetting Sheet by Sheet; New Data is SQL, Vetter Pulled Their Own Data

In [ ]:
import pandas as pd
import numpy as np
import sqlite3

Pulling Your Own Data Quickly (if you do not have your own pulled data)

In [ ]:
df = pd.read_excel(r"C:\Users\Norah\Downloads\efotw-2024-master-index-data-for-researchers-iso.xlsx", sheet_name="Sheet1")
country = pd.read_excel(r"C:\Users\Norah\Downloads\efotw-2024-master-index-data-for-researchers-iso.xlsx", sheet_name="Sheet3")
df = df.pivot_table(index='Countries', columns='Year', values='Value')
dt = country.merge(df, on='Countries', how='left')
dt = dt.drop(columns=['Countries'])
year_columns = [col for col in dt.columns if str(col).isdigit()]

dt['Earliest'] = dt[year_columns].bfill(axis=1).iloc[:, 0]
dt['MostRecent'] = dt[year_columns].ffill(axis=1).iloc[:, -1]
dt[year_columns + ['Earliest', 'MostRecent']] = dt[year_columns + ['Earliest', 'MostRecent']].round(4)

dt

In [ ]:
#data pullers data
con = sqlite3.connect(r"C:\Users\Norah\Downloads\IFsDataImport (8).db")
sql_query = """SELECT name FROM sqlite_master  
  WHERE type='table';"""
cursor = con.cursor()

cursor.execute(sql_query)

In [ ]:
tbname = 'SeriesFreedomEcon'

dt_puller = pd.read_sql_query(f"SELECT * FROM '{tbname}'", con)
col = dt_puller.columns[dt_puller.dtypes.eq('float64')]

#you can change between an excel if you already have one or if you use the above code to create a new one use dt 

#dt = pd.read_excel(r'C:\Users\Norah\OneDrive\Documents\ilogender.xlsx',sheet_name='Utlities_Male')
dt_vetter = dt.melt(id_vars= ["Country","FIPS_CODE"],var_name="Year",value_name="Vetter_Value")

dt_puller = dt_puller.melt(id_vars= ["Country","FIPS_CODE"],var_name="Year",value_name="Puller_Value")

dt_vetter = dt_vetter.sort_values(by=['Country', 'FIPS_CODE', 'Year']).reset_index(drop=True)
dt_puller = dt_puller.sort_values(by=['Country', 'FIPS_CODE', 'Year']).reset_index(drop=True)


dt_puller.Year = dt_puller.Year.apply(pd.to_numeric, errors='coerce')
dt_vetter.Year = dt_vetter.Year.apply(pd.to_numeric, errors='coerce')
dt_puller.Puller_Value = dt_puller.Puller_Value.apply(pd.to_numeric, errors='coerce')
dt_vetter.Vetter_Value = dt_vetter.Vetter_Value.apply(pd.to_numeric, errors='coerce')

compare = dt_vetter.merge(
                       dt_puller, 
                       left_on =[
                                 "Country",
                                 "FIPS_CODE",
                                 "Year",
                                 ], 
                       right_on =[
                                  "Country",
                                  "FIPS_CODE",
                                  "Year",
                                  ],
                       how = 'outer', 
                       )
compare = compare[~compare[['Puller_Value','Vetter_Value']].isna().all(axis=1)]
compare = compare[compare['Year'] <= 2027]
compare['check'] = abs(compare['Puller_Value'] - compare['Vetter_Value'])/compare['Vetter_Value']
compare[((compare['check'] >= 0.15) | (compare['check'].isnull()))
        ]
compare

# can check for N/A values or values above a certain threshold like 0 
#comparech = compare[compare['check'].isna()]
#comparech = compare[compare['check'] > 0]
#comparech

# to Excel
#compare.to_csv('GDPcomparison2011new.csv')

If you want to see full list in python itself and not export to excel

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

compare